# Input-output properties cells

In [1]:
# %%capture
# !pip install tmd wget neurom[plotly] ipywidgets

In [2]:
import numpy as np
import json
import os
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# CircuitPath = 'O1_data_physiology/' # edges files not inclued in https://github.com/FernandoSBorges/NetPyNE_SONATA/tree/main/SCx_model/O1_data_physiology

CircuitPath = '/home/fernando/Dropbox/SUNY/2025/TACS/SCx_model/O1_data_physiology/'
Atlas = CircuitPath + 'atlas/'
Atlas2 = 'O1_data_physiology/voxel_atlas_data/'
MorphologyPath = CircuitPath + 'morphologies/ascii/'
nrnPath = CircuitPath + 'S1nonbarrel_neurons__S1nonbarrel_neurons__chemical/edges.h5'
CellLibraryFile = CircuitPath + 'S1nonbarrel_neurons/nodes.h5'
METypePath = CircuitPath + 'emodels_hoc/'

lst_properties = [ 'etype', 'exc_mini_frequency', 'inh_mini_frequency', 'layer', 'me_combo', 'model_template', 'model_type', 'morph_class', 'morphology', 'mtype', 
                  'orientation_w', 'orientation_x', 'orientation_y', 'orientation_z', 'population', 'region', 'synapse_class', '@dynamics:holding_current', 
                  '@dynamics:input_resistance', '@dynamics:resting_potential', '@dynamics:threshold_current', 'x', 'y', 'z']

In [4]:
from bluepysnap import Circuit
from bluepysnap.bbp import Cell
circuit_path = CircuitPath + 'circuit_config.json'
circuit = Circuit(circuit_path)

cells = circuit.nodes["S1nonbarrel_neurons"]
proj_cells = circuit.nodes["external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical"]
cells_projections = proj_cells
nodesinfo = cells.get()
nodesinfo_projections = cells_projections.get()

In [5]:
nodesinfo[nodesinfo['layer'] == '4'].head()

,etype,exc_mini_frequency,hexagon,inh_mini_frequency,layer,me_combo,model_template,model_type,morph_class,morphology,...,x,x_new,y,y_new,z,z_new,@dynamics:holding_current,@dynamics:input_resistance,@dynamics:resting_potential,@dynamics:threshold_current
node_ids,,,,,,,,,,,,,,,,,,,,,
108727,dSTUT,0.010108,2,0.233243,4,dSTUT_L4BP_L4_BP_4_C300797C-I1_-_Scale_x1.000_...,hoc:dSTUT_L4BP,biophysical,INT,C300797C-I1_-_Scale_x1.000_y0.975_z1.000_-_Clo...,...,3918.383222,345.100991,-882.079924,368.523207,-1946.094942,1431.980963,-0.048744,103.355850,-77.048584,0.092756
108728,bAC,0.010108,1,0.233243,4,bAC_L23BTC_L4_BP_4_rp140319_ChC_3_idA_-_Scale_...,hoc:bAC_L23BTC,biophysical,INT,rp140319_ChC_3_idA_-_Scale_x1.000_y0.975_z1.00...,...,4140.810336,179.239589,-766.305463,605.707003,-2090.662966,1435.342921,-0.011682,606.258118,-76.392754,0.024155
108729,cNAC,0.010108,0,0.233243,4,cNAC_L6NGC_L4_BP_4_C300797C-I1_-_Scale_x1.000_...,hoc:cNAC_L6NGC,biophysical,INT,C300797C-I1_-_Scale_x1.000_y0.950_z1.000_-_Clo...,...,3953.899058,722.166787,-1279.341226,593.262166,-2143.970161,1357.507449,-0.025396,385.013672,-74.105980,0.033770
108730,cACint,0.010108,0,0.233243,4,cACint_L4CHC_L4_BP_4_rp140319_ChC_3_idA_-_Scal...,hoc:cACint_L4CHC,biophysical,INT,rp140319_ChC_3_idA_-_Scale_x1.000_y0.975_z1.00...,...,4214.386218,740.411420,-1362.666051,842.778874,-2201.251811,1481.951494,-0.014442,636.180054,-73.075333,0.015156
108731,cACint,0.010108,0,0.233243,4,cACint_L4CHC_L4_BP_4_rp140319_ChC_3_idA_-_Scal...,hoc:cACint_L4CHC,biophysical,INT,rp140319_ChC_3_idA_-_Scale_x1.000_y1.025_z1.00...,...,4184.420803,714.604811,-1328.656268,916.399225,-2366.952165,1328.898361,-0.014949,620.984924,-72.957344,0.015535


In [6]:
def distance3D(gidpre,gidpost):
    return np.sqrt(np.power(nodesinfo['x'][gidpre]-nodesinfo['x'][gidpost],2)+np.power(nodesinfo['y'][gidpre]-nodesinfo['y'][gidpost],2)+np.power(nodesinfo['z'][gidpre]-nodesinfo['z'][gidpost],2))

def distance2D(gidpre,gidpost):
    return np.sqrt(np.power(nodesinfo['x'][gidpre]-nodesinfo['x'][gidpost],2)+np.power(nodesinfo['y'][gidpre]-nodesinfo['y'][gidpost],2))

def distance2Dmean(gidpre, mean_x, mean_y):
    return np.sqrt(np.power(nodesinfo['x_new'][gidpre]-mean_x,2)+np.power(nodesinfo['y_new'][gidpre]-mean_y,2))

In [15]:
print(cells.property_values(Cell.REGION))

mtypes = sorted(cells.property_values(Cell.MTYPE))

i = 0
mntypes = {}
for mn in sorted(mtypes):
    mntypes[mn] = i
    i+=1

{'S1FL', 'S1J', 'S1DZ'}


In [7]:
f = open('node_sets.json') 
node_sets = json.load(f) 

mean_x, mean_y = np.mean(nodesinfo['x_new']), np.mean(nodesinfo['y_new'])
node_gid = [] 
nodepremtype_new = []    
mtype_new = []     

# for gid in node_sets['hex0']['node_id']: 
#     if 'L1' in nodesinfo['mtype'][gid] and distance2Dmean(gid, mean_x, mean_y)< 40.0:  
#         node_gid.append(gid)
#         nodepremtype_new.append(mntypes[nodesinfo['mtype'][gid]])
#         if nodesinfo['mtype'][gid] not in mtype_new:
#             mtype_new.append(nodesinfo['mtype'][gid])

#         print(len(node_gid),gid,nodesinfo['mtype'][gid])
        
# for gid in node_sets['hex0']['node_id']: 
#     if 'L2' in nodesinfo['mtype'][gid] and distance2Dmean(gid, mean_x, mean_y)< 8.0:  
#         node_gid.append(gid)
#         nodepremtype_new.append(mntypes[nodesinfo['mtype'][gid]])
#         if nodesinfo['mtype'][gid] not in mtype_new:
#             mtype_new.append(nodesinfo['mtype'][gid])

#         print(len(node_gid),gid,nodesinfo['mtype'][gid])

# for gid in node_sets['hex0']['node_id']: 
#     if 'L3_' in nodesinfo['mtype'][gid] and distance2Dmean(gid, mean_x, mean_y)< 4.0:  
#         node_gid.append(gid)
#         nodepremtype_new.append(mntypes[nodesinfo['mtype'][gid]])
#         if nodesinfo['mtype'][gid] not in mtype_new:
#             mtype_new.append(nodesinfo['mtype'][gid])

#         print(len(node_gid),gid,nodesinfo['mtype'][gid])
        
# for gid in node_sets['hex0']['node_id']: 
#     if 'L4' in nodesinfo['mtype'][gid] and distance2Dmean(gid, mean_x, mean_y)< 3.5:  
#         node_gid.append(gid)
#         nodepremtype_new.append(mntypes[nodesinfo['mtype'][gid]])
#         if nodesinfo['mtype'][gid] not in mtype_new:
#             mtype_new.append(nodesinfo['mtype'][gid])

#         print(len(node_gid),gid,nodesinfo['mtype'][gid])
            
# for gid in node_sets['hex0']['node_id']: 
#     if 'L5' in nodesinfo['mtype'][gid] and distance2Dmean(gid, mean_x, mean_y)< 4.5:  
#         node_gid.append(gid)
#         nodepremtype_new.append(mntypes[nodesinfo['mtype'][gid]])
#         if nodesinfo['mtype'][gid] not in mtype_new:
#             mtype_new.append(nodesinfo['mtype'][gid])

#         print(len(node_gid),gid,nodesinfo['mtype'][gid])
        
# for gid in node_sets['hex0']['node_id']: 
#     if 'L6' in nodesinfo['mtype'][gid] and distance2Dmean(gid, mean_x, mean_y)< 3.5:  
#         node_gid.append(gid)
#         nodepremtype_new.append(mntypes[nodesinfo['mtype'][gid]])
#         if nodesinfo['mtype'][gid] not in mtype_new:
#             mtype_new.append(nodesinfo['mtype'][gid])

#         print(len(node_gid),gid,nodesinfo['mtype'][gid])

# # non central col

# for hex in node_sets['hex_O1']:
#     for gid in node_sets[hex]['node_id']:     
#         if 'L1_' in nodesinfo['mtype'][gid] and distance2Dmean(gid, mean_x, mean_y) > 703.0 and distance2Dmean(gid, mean_x, mean_y) < 704.0:  
#             node_gid.append(gid)
#             nodepremtype_new.append(mntypes[nodesinfo['mtype'][gid]])
#             if nodesinfo['mtype'][gid] not in mtype_new:
#                 mtype_new.append(nodesinfo['mtype'][gid])

#             print(len(node_gid),gid,nodesinfo['mtype'][gid],hex,distance2Dmean(gid, mean_x, mean_y))        
            
for hex in node_sets['hex_O1']:
    for gid in node_sets[hex]['node_id']:     
        if 'L6_TPC' in nodesinfo['mtype'][gid] and distance2Dmean(gid, mean_x, mean_y) > 600.0 and distance2Dmean(gid, mean_x, mean_y) < 600.1:  
            node_gid.append(gid)
            nodepremtype_new.append(mntypes[nodesinfo['mtype'][gid]])
            if nodesinfo['mtype'][gid] not in mtype_new:
                mtype_new.append(nodesinfo['mtype'][gid])

            print(len(node_gid),gid,nodesinfo['mtype'][gid],hex,distance2Dmean(gid, mean_x, mean_y))        

# mtype_new, distance3D(90290,96883), distance3D(90290,102178)

NameError: name 'mntypes' is not defined

In [17]:
f = open('node_sets.json') 
node_sets = json.load(f) 

mean_x, mean_y = np.mean(nodesinfo['x_new']), np.mean(nodesinfo['y_new'])
node_gid = [] 
nodepremtype_new = []    
mtype_new = []     

hex = 'hex0'
for gid in node_sets['hex0']['node_id']: 
    if 'L1' in nodesinfo['mtype'][gid] and distance2Dmean(gid, mean_x, mean_y)< 40.0:  
        node_gid.append(gid)
        nodepremtype_new.append(mntypes[nodesinfo['mtype'][gid]])
        if nodesinfo['mtype'][gid] not in mtype_new:
            mtype_new.append(nodesinfo['mtype'][gid])

        print(len(node_gid),gid,nodesinfo['mtype'][gid],hex,distance2Dmean(gid, mean_x, mean_y))
        
for gid in node_sets['hex0']['node_id']: 
    if 'L23' in nodesinfo['mtype'][gid] and distance2Dmean(gid, mean_x, mean_y)< 25.0:  
        node_gid.append(gid)
        nodepremtype_new.append(mntypes[nodesinfo['mtype'][gid]])
        if nodesinfo['mtype'][gid] not in mtype_new:
            mtype_new.append(nodesinfo['mtype'][gid])

        print(len(node_gid),gid,nodesinfo['mtype'][gid],hex,distance2Dmean(gid, mean_x, mean_y))

for gid in node_sets['hex0']['node_id']: 
    if 'L2_' in nodesinfo['mtype'][gid] and distance2Dmean(gid, mean_x, mean_y)< 25.0:  
        node_gid.append(gid)
        nodepremtype_new.append(mntypes[nodesinfo['mtype'][gid]])
        if nodesinfo['mtype'][gid] not in mtype_new:
            mtype_new.append(nodesinfo['mtype'][gid])

        print(len(node_gid),gid,nodesinfo['mtype'][gid],hex,distance2Dmean(gid, mean_x, mean_y))

for gid in node_sets['hex0']['node_id']: 
    if 'L3_' in nodesinfo['mtype'][gid] and distance2Dmean(gid, mean_x, mean_y)< 25.0:  
        node_gid.append(gid)
        nodepremtype_new.append(mntypes[nodesinfo['mtype'][gid]])
        if nodesinfo['mtype'][gid] not in mtype_new:
            mtype_new.append(nodesinfo['mtype'][gid])

        print(len(node_gid),gid,nodesinfo['mtype'][gid],hex,distance2Dmean(gid, mean_x, mean_y))
        
for gid in node_sets['hex0']['node_id']: 
    if 'L4' in nodesinfo['mtype'][gid] and distance2Dmean(gid, mean_x, mean_y)< 25.0:  
        node_gid.append(gid)
        nodepremtype_new.append(mntypes[nodesinfo['mtype'][gid]])
        if nodesinfo['mtype'][gid] not in mtype_new:
            mtype_new.append(nodesinfo['mtype'][gid])

        print(len(node_gid),gid,nodesinfo['mtype'][gid],hex,distance2Dmean(gid, mean_x, mean_y))
            
for gid in node_sets['hex0']['node_id']: 
    if 'L5' in nodesinfo['mtype'][gid] and distance2Dmean(gid, mean_x, mean_y)< 25.0:  
        node_gid.append(gid)
        nodepremtype_new.append(mntypes[nodesinfo['mtype'][gid]])
        if nodesinfo['mtype'][gid] not in mtype_new:
            mtype_new.append(nodesinfo['mtype'][gid])

        print(len(node_gid),gid,nodesinfo['mtype'][gid],hex,distance2Dmean(gid, mean_x, mean_y))
        
for gid in node_sets['hex0']['node_id']: 
    if 'L6' in nodesinfo['mtype'][gid] and distance2Dmean(gid, mean_x, mean_y)< 25.0:  
        node_gid.append(gid)
        nodepremtype_new.append(mntypes[nodesinfo['mtype'][gid]])
        if nodesinfo['mtype'][gid] not in mtype_new:
            mtype_new.append(nodesinfo['mtype'][gid])

        print(len(node_gid),gid,nodesinfo['mtype'][gid],hex,distance2Dmean(gid, mean_x, mean_y))

# non central col

# for hex in node_sets['hex_O1']:
#     for gid in node_sets[hex]['node_id']:     
#         if 'L1_' in nodesinfo['mtype'][gid] and distance2Dmean(gid, mean_x, mean_y) > 703.0 and distance2Dmean(gid, mean_x, mean_y) < 704.0:  
#             node_gid.append(gid)
#             nodepremtype_new.append(mntypes[nodesinfo['mtype'][gid]])
#             if nodesinfo['mtype'][gid] not in mtype_new:
#                 mtype_new.append(nodesinfo['mtype'][gid])

#             print(len(node_gid),gid,nodesinfo['mtype'][gid],hex,distance2Dmean(gid, mean_x, mean_y))        
            
# for hex in node_sets['hex_O1']:
#     for gid in node_sets[hex]['node_id']:     
#         if 'L6_TPC' in nodesinfo['mtype'][gid] and distance2Dmean(gid, mean_x, mean_y) > 600.0 and distance2Dmean(gid, mean_x, mean_y) < 600.1:  
#             node_gid.append(gid)
#             nodepremtype_new.append(mntypes[nodesinfo['mtype'][gid]])
#             if nodesinfo['mtype'][gid] not in mtype_new:
#                 mtype_new.append(nodesinfo['mtype'][gid])

#             print(len(node_gid),gid,nodesinfo['mtype'][gid],hex,distance2Dmean(gid, mean_x, mean_y))        

# mtype_new, distance3D(90290,96883), distance3D(90290,102178)

1 7 L1_DAC hex0 26.990737576695835
2 33788 L1_NGC-DA hex0 38.63732975121551
3 34575 L1_HAC hex0 38.85416462870949
4 35137 L1_SAC hex0 38.683540484060075
5 35157 L1_SAC hex0 32.63913689668969
6 35206 L1_SAC hex0 39.62204687171059
7 26321 L23_MC hex0 22.096084070034653
8 147807 L23_NGC hex0 6.471895101606248
9 148107 L23_BTC hex0 17.91359872640971
10 148175 L23_BTC hex0 17.449080378355337
11 150634 L23_NBC hex0 16.330683341644516
12 151258 L23_NBC hex0 15.757094597291808
13 143026 L2_TPC:A hex0 19.075714695829436
14 143162 L2_TPC:A hex0 23.169094759918117
15 143288 L2_TPC:A hex0 21.871511005386537
16 144429 L2_TPC:A hex0 20.239879500541257
17 144555 L2_TPC:A hex0 22.288160693365704
18 145753 L2_TPC:A hex0 22.306682397157093
19 146854 L2_IPC hex0 7.792017276943925
20 147548 L2_IPC hex0 15.317250383105312
21 152318 L2_TPC:B hex0 20.99192853390441
22 152556 L2_TPC:B hex0 20.419154489940226
23 154199 L2_TPC:B hex0 8.11713170477625
24 155400 L2_TPC:B hex0 10.258436971417288
25 155672 L2_TPC:B

In [18]:
cellName_list = {}
cellName_list2 = []
gid_list = {}

for gid in node_gid:
    MorphoName = nodesinfo['morphology'][gid] + '.asc'
    hocName = nodesinfo['model_template'][gid][4:]  
    mcName = nodesinfo['region'][gid][:3]  
    Mtype = nodesinfo['mtype'][gid]
    cellName = nodesinfo['mtype'][gid] + '_' + nodesinfo['etype'][gid] + '_' + nodesinfo['region'][gid][:3]
    MEName = nodesinfo['mtype'][gid] + '_' + nodesinfo['etype'][gid]
    MorphohocName = nodesinfo['morphology'][gid] + '__' + nodesinfo['model_template'][gid][4:]    

    if cellName not in cellName_list2:
        cellName_list2 = []
        
    cellName_list2.append(cellName)
    
    cellName_list[gid] = cellName + '_' + str(len(cellName_list2)-1)
    gid_list[cellName + '_' + str(len(cellName_list2)-1)] = gid
    # print('%s \n %d %s %s \n hoc = %s \n asc = %s' % (cellName,gid,mcName,Mtype,hocName,MorphoName))
    # print('%s \t %d %s %s \t hoc = %s \t asc = %s' % (cellName,gid,mcName,Mtype,hocName,MorphoName))
    
# cellName_list

In [19]:
import neurom, scipy
from neurom import geom
from neurom.core.morphology import iter_neurites
from neurom import load_morphologies
# # from neurom import viewer
# import os
# import wget
# import tmd

# import neurom as nm
# import ipywidgets as widgets

from neurom.view import plotly_impl

# import os

# from neuron import h
# from neuron.units import mV, ms
# import plotly
# import plotly.graph_objects as go

def transform_neuron(nrn_morph, nodesinfo):
    rot = scipy.spatial.transform.Rotation.from_quat(nodesinfo[["orientation_x", "orientation_y",
                                                                 "orientation_z", "orientation_w"]].values[gid])
    rot = neurom.geom.transform.Rotation(rot.as_matrix())
    tl = neurom.geom.transform.Translation(nodesinfo[["x", "y", "z"]].values[gid] - nodesinfo[["x", "y", "z"]].values[gid])
    return nrn_morph.transform(rot).transform(tl)
    
def load_neuron(nodesinfo, transform=True):
    
    fn = MorphologyPath + nodesinfo['morphology'][gid] + '.asc'
    
    nrn = neurom.load_morphology(fn)
    if transform:
        nrn = transform_neuron(nrn, nodesinfo)
    return nrn

In [20]:
for cellName in sorted(gid_list.keys()):
    
    gid = gid_list[cellName]

    print(cellName,gid)

    fn = MorphologyPath + nodesinfo['morphology'][gid] + '.asc'

    fn_new = '/home/fernando/Documents/SCx_model/O1_data_physiology/morphologies2/ascii/' + nodesinfo['morphology'][gid] + "_gid" + str(gid) + ".asc"

    if nodesinfo['morphology'][gid] + "_gid" + str(gid) + ".asc" in os.listdir('/home/fernando/Documents/SCx_model/O1_data_physiology/morphologies2/ascii/'):
        print('file exist',fn_new)
    else:
        print('creating',fn_new)
    
        nrn_morph = load_neuron(nodesinfo, transform=False)
        nrn_morph_new = load_neuron(nodesinfo, transform=True)
        # fig, ax = plotly_impl.plot_morph3d(nrn_morph, inline=True)

        with open(fn, 'r') as f:
            morpho_asc = f.read()

        n_points = nrn_morph.points
        n_points_new = nrn_morph_new.points
        for ii, neurites in enumerate(n_points):
            morpho_asc = morpho_asc.replace( "%.9f %.9f %.9f" % (n_points[ii][0],n_points[ii][1],n_points[ii][2]), "%.9f %.9f %.9f" % (n_points_new[ii][0],n_points_new[ii][1],n_points_new[ii][2]))

        # n_points = nrn_morph.soma.points
        # n_points_new = nrn_morph_new.soma.points
        # for ii, neurites in enumerate(n_points):
        #     morpho_asc = morpho_asc.replace( "%.9f %.9f %.9f" % (n_points[ii][0],n_points[ii][1],n_points[ii][2]), "%.9f %.9f %.9f" % (n_points_new[ii][0],n_points_new[ii][1],n_points_new[ii][2]))

        with open(fn_new, 'w') as f:
                f.write(morpho_asc)


L1_DAC_cNAC_S1F_0 7


FileNotFoundError: [Errno 2] No such file or directory: '/home/fernando/Documents/SCx_model/O1_data_physiology/morphologies2/ascii/'

In [12]:
# for cellName in sorted(gid_list.keys())[0:1]:
    
#     gid = gid_list[cellName]

for gid in [38662, 88937]: # 7, 147807, 169225, 22787, 115007, 135956, 186706, 189951, 38662, 88937]:    
    
    # nrn_morph = load_neuron(nodesinfo, transform=False)
    # fig, ax = plotly_impl.plot_morph3d(nrn_morph, inline=True)
    nrn_morph_new = load_neuron(nodesinfo, transform=True)
    fig, ax = plotly_impl.plot_morph3d(nrn_morph_new, inline=False);